# Day 16

link: https://adventofcode.com/2022/day/16

ข้อนี้เป็นโจทย์กราฟ valve แต่ละอันก็คือ node ของกราฟ input ของเรามี 63 valves แต่พอดูดีๆ แล้วจะพบว่า
valve จำนวนมากมี flow rate เป็น 0 และมีอุโมงค์เชื่อมกับ valve อื่นแค่ 2 อัน
ดังนั้น valve เหล่านี้เป็นแค่ทางผ่าน ทำหน้าที่แค่เพิ่มระยะเวลาในการเดินทางระหว่าง valve ที่มี flow rate เป็นบวกเท่านั้น

เราเริ่มจากอ่าน input แล้วเก็บเป็น hash ของ node ไว้ โดยที่แต่ละ node จะเก็บ cost ของการเดินทางไป node ที่อยู่ติดกัน เริ่มต้นเป็น 1 นาที
จากนั้นเราจะเอา node ทางผ่านออก แล้วไปเพิ่ม cost ให้ node ข้างๆ แทน
step นี้ลดจำนวน node ลงมาเหลือแค่ 16

In [1]:
input = IO.foreach('in16.txt').to_a.map(&:strip)

valves = {}
input.each{|line|
  words = line.split(/\,?[\s;=]/)
  valve_name = words[1]
  flow = words[5].to_i
  adjacents = words[11..-1].map{|n| [n, 1]}.to_h
  valves[valve_name] = [flow, adjacents]
}

def collapse_tunnel(valves, v)
  target_valve = valves[v]
  adj1, adj2 = target_valve[1].keys
  adj1_valve = valves[adj1]
  adj2_valve = valves[adj2]
  new_cost = adj1_valve[1][v] + target_valve[1][adj2]
  
  adj1_valve[1].delete(v)
  adj1_valve[1][adj2] = new_cost
  
  adj2_valve[1].delete(v)
  adj2_valve[1][adj1] = new_cost
  
  valves.delete(v)
end

valves.keys.to_a.each{|v|
  flow, adjacents = valves[v]
  collapse_tunnel(valves, v) if flow == 0 && adjacents.size == 2
}

nil

## Part 1

โจทย์บอกว่าเรามีเวลา 30 นาที ให้พยายามไปเปิด valve ให้ pressure ออกมามากที่สุด

เราจะใช้วิธีค่อนข้าง brute force คือวนทุกลำดับของ valve ที่เป็นไปได้ แล้วดูว่าลำดับไหนได้ pressure เยอะที่สุด
ลองประเมินคร่าวๆ เรามี valve อยู่ 15 อัน (ไม่รวมจุดเริ่มต้นที่ valve AA) ถ้าวนทุก order จะต้องลองทั้งหมด 15! ซึ่งเยอะไปมาก
แต่เนื่องจากเวลามีจำกัดแค่ 30 นาที และเราต้องใช้เวลาอย่างน้อย 3 นาทีต่อ valve (เวลาเปิด 1 นาที. เวลาเดินทางอย่างน้อย 2 นาที)
ดังนั้นเราเปิดได้มากที่สุดแค่ 10 valve เพราะฉะนั้นมีความเป็นไปได้อยู่ไม่เกิน 10! ~ 3M ซึ่งเล็กพอที่เราจะวนทุกรูปแบบได้

ก่อนอื่นเราหา shortest path length ของทุกๆ คู่ของ valve โดยใช้ Floyd-Warshall
จากนั้นก็ recursive backtrack เพื่อคำนวณ pressure โดยจะหยุดแต่ก tree แล้ว backtrack เมื่อ:
1. หมดเวลา
2. valve ถูกเปิดครบแล้ว (ไม่ได้ใช้ใน part นี้ แต่จะต้องใช้สำหรับโจทย์ตัวอย่าง และ part หน้า)

ใน implementation จริงเราเพิ่ม memoization ซึ่งลดการคำนวณซ้ำซ้อนได้ไม่น้อย

(โค้ดนี้ทำไว้สำหรับ part 2 ด้วย สำหรับครึ่งแรกนี้ เรามี agent แค่คนเดียว ดังนั้น `remain_agents` จะเป็น 1 ตลอด ซึ่งทำให้ `next_agent` เป็น 0 เสมอ)

In [2]:
valve_names = valves.keys.to_a.sort
valve_lookup = valve_names.each_with_index.to_h
flow_rates = valve_names.map{|v| valves[v][0]}

valve_count = valve_names.size

inf = 10**10
dist = valve_count.times.map{Array.new(valve_count, inf)}
valve_names.each_with_index{|v, i|
  dist[i][i] = 0
  valves[v][1].each{|w, d|
    dist[i][valve_lookup[w]] = d
  }
}
valve_count.times{|k|
  valve_count.times{|i|
    valve_count.times{|j|
      ikj = dist[i][k] + dist[k][j]
      dist[i][j] = ikj if dist[i][j] > ikj 
    }
  }
}

class Calculator

  @@memo = {}

  def initialize(flow_rates, dist, remain_minutes)
    @flow_rates = flow_rates
    @dist = dist
    @remain_minutes = remain_minutes
  end

  def traverse(current_valve, remain_valves, remain_minutes, remain_agents)
    return 0 if remain_agents == 0
    return 0 if remain_valves.empty?
    
    memkey = [current_valve, remain_valves, remain_minutes, remain_agents]
    return @@memo[memkey] if @@memo[memkey]

    next_agent = traverse(0, remain_valves, @remain_minutes, remain_agents - 1)
    this_agent = remain_valves.size.times.map{|i|
      v = remain_valves[i]
      new_remain_minutes = remain_minutes - 1 - @dist[current_valve][v]
      if new_remain_minutes > 0
        spent = new_remain_minutes * @flow_rates[v]
        new_remain_valves = remain_valves.clone
        new_remain_valves.delete_at(i)
        traverse(
          v, 
          new_remain_valves, 
          new_remain_minutes,
          remain_agents
        ) + spent
      else
        0
      end
    }.max
    best = [next_agent, this_agent].max
    @@memo[memkey] = best
    best
    
  end

  def best_pressure_relieved(agents)
    traverse(0, (1...@flow_rates.size).to_a, @remain_minutes, agents)
  end

end

calc1 = Calculator.new(flow_rates, dist, 30)
puts calc1.best_pressure_relieved(1)

1796


## Part 2

โจทย์ครึ่งหลังลดเวลาลงเหลือ 26 นาที แต่เพิ่มจำนวน agent ที่ไปเปิด valve เป็น 2 คน เราเอา backtracking จากครึ่งแรกมาแก้เล็กน้อย
(ซึ่งใน code ข้างบนก็คือรวมส่วนที่แก้ไปแล้ว) จุดสำคัญคือ agent ทั้ง 2 จะไม่เปิด valve ซ้ำกัน
เราใช้วิธี backtrack เหมือนเดิมสำหรับ agent คนแรก แต่เราสามารถให้ agent คนแรกนี้หยุดทำงานได้ทุกเมื่อ แล้วส่งต่อ valve ที่เหลือให้ agent คนที่สอง ซึ่งจะกลับไปเริ่มต้นที่เวลา 26 นาที ทำ backtrack ต่อด้วย valve ที่เหลือ

In [3]:
calc2 = Calculator.new(flow_rates, dist, 26)
puts calc2.best_pressure_relieved(2)

1999


ที่จริงยังมีอีกท่านึงที่เราลอง คือเราใช้วิธีคิด pressure จากทุกๆ partition ของ valves ที่เป็นไปได้ (ซึ่งมีทั้งหมด 2^15/2 = 16K partitions) แล้วหาคู่ที่รวม pressure ได้สูงที่สุด แต่ลองแล้ววิธีนี้ใช้เวลาเยอะกว่า